# Basic indexing and searching with RAGatouille

In this quick example, we'll use the `RAGPretrainedModel` magic class to demonstrate how to:

- **Build an index from raw documents**
- **Search an index for relevant documents**
- **Load an index and the associated pretrained model to update or query it.**

Please note: Indexing is currently not supported on Google Colab and Windows 10.

First, let's load up a pre-trained ColBERT model:

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


And that's all you need to do to load the model! All the config is now stored, and ready to be used for indexing.

## Creating an index

Let's index some documents now. We'll use data from Wikipedia, to build our Miyazaki-Index, which will store all you could ever know about Hayao Miyazaki('s wikipedia page).

First, let's write a function to fetch the data from the Wikipedia with a clear user-agent, to be a good netizen:

In [7]:
import requests

def get_wikipedia_page(title: str):
    """
    Retrieve the full text content of a Wikipedia page.
    
    :param title: str - Title of the Wikipedia page.
    :return: str - Full text content of the page as raw string.
    """
    # Wikipedia API endpoint
    URL = "https://en.wikipedia.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts",
        "explaintext": True,
    }

    # Custom User-Agent header to comply with Wikipedia's best practices
    headers = {
        "User-Agent": "RAGatouille_tutorial/0.0.1 (ben@clavie.eu)"
    }

    response = requests.get(URL, params=params, headers=headers)
    data = response.json()

    # Extracting page content
    page = next(iter(data['query']['pages'].values()))
    return page['extract'] if 'extract' in page else None

And now, let's use it to fetch the page's content and check how long it is:

In [8]:
full_document = get_wikipedia_page("Hayao_Miyazaki")
len(full_document)

45346

That's a lot of characters! Thankfully, `RAGPretrainedColBERT.index()` also relies on a `CorpusProcessor`! It takes in various pre-processing functions and applies them to your documents before embedding and indexing them.

By default, `CorpusProcessor` uses LlamaIndex's `SentenceSplitter`, with a chunk-size defined by your index's max document length. By default, `max_document_length` is 256 tokens, but you can set it to whatever you like.

Let's keep our information units small and go for 180 when creating our index. We'll also add an optional document ID and an optional metadata entry for our index:

In [9]:
RAG.index(
    collection=[full_document], 
    document_ids=['miyazaki'],
    document_metadatas=[{"entity": "person", "source": "wikipedia"}],
    index_name="Miyazaki", 
    max_document_length=180, 
    split_documents=True
    )



[Jan 25, 23:29:10] #> Note: Output directory .ragatouille/colbert/indexes/Miyazaki already exists


[Jan 25, 23:29:10] #> Will delete 11 files already at .ragatouille/colbert/indexes/Miyazaki in 20 seconds...
#> Starting...
[Jan 25, 23:29:34] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/2 [00:00<?, ?it/s]/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Jan 25, 23:29:35] [0] 		 #> Encoding 81 passages..


 50%|█████     | 1/2 [00:02<00:02,  2.65s/it]/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 2/2 [00:03<00:00,  1.65s/it]
WARNING clustering 9984 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]

[Jan 25, 23:29:38] [0] 		 avg_doclen_est = 129.74073791503906 	 len(local_sample) = 81
[Jan 25, 23:29:38] [0] 		 Creating 1,024 partitions.
[Jan 25, 23:29:38] [0] 		 *Estimated* 10,508 embeddings.
[Jan 25, 23:29:38] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki/plan.json ..
Clustering 9984 points in 128D to 1024 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
[0.035, 0.04, 0.039, 0.035, 0.034, 0.037, 0.036, 0.033, 0.036, 0.039, 0.035, 0.038, 0.034, 0.039, 0.036, 0.037, 0.032, 0.036, 0.035, 0.038, 0.037, 0.037, 0.036, 0.037, 0.036, 0.033, 0.039, 0.033, 0.037, 0.036, 0.038, 0.039, 0.039, 0.034, 0.034, 0.034, 0.035, 0.038, 0.036, 0.04, 0.037, 0.037, 0.034, 0.033, 0.037, 0.033, 0.036, 0.038, 0.037, 0.036, 0.034, 0.035, 0.036, 0.036, 0.038, 0.037, 0.038, 0.036, 0.038, 0.035, 0.034, 0.036, 0.033, 0.034, 0.038, 0.034, 0.037, 0.038, 0.034, 0.034, 0.037, 0.036, 0.034, 0.038, 0.036, 0.034, 0.034, 0.037, 0.036, 0.035, 0.039, 0.038, 0.034, 0.039, 0.034


100%|██████████| 2/2 [00:02<00:00,  1.43s/it]
1it [00:02,  2.89s/it]
100%|██████████| 1024/1024 [00:00<00:00, 311522.98it/s]


[Jan 25, 23:29:42] #> Optimizing IVF to store map from centroids to list of pids..
[Jan 25, 23:29:42] #> Building the emb2pid mapping..
[Jan 25, 23:29:42] len(emb2pid) = 10509
[Jan 25, 23:29:42] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki/ivf.pid.pt

#> Joined...
Done indexing!


'.ragatouille/colbert/indexes/Miyazaki'

And that's our index created! It's already compressed and save to disk, so you're ready to use it anywhere you want. By the way, the default behaviour of `index()` is to split documents, but if for any reason you'd like them to remain intact (if you've already preprocessed them, for example), you can set it to false to bypass it!

Let's move on to querying our index now...

## Retrieving Documents

`RAGPretrainedModel` has just indexed our document, so the index is already loaded into it and ready to use! 

Searching is very simple and straightforward, let's say I have a single query:

In [10]:
k = 3 # How many documents you want to retrieve, defaults to 10, we set it to 3 here for readability
results = RAG.search(query="What animation studio did Miyazaki found?", k=k)
results

Loading searcher for index Miyazaki for the first time... This may take a few seconds
[Jan 25, 23:29:45] #> Loading codec...
[Jan 25, 23:29:45] #> Loading IVF...
[Jan 25, 23:29:45] Loading segmented_lookup_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Jan 25, 23:29:46] #> Loading doclens...


100%|███████████████████████| 1/1 [00:00<00:00, 6177.18it/s]

[Jan 25, 23:29:46] #> Loading codes and residuals...



100%|████████████████████████| 1/1 [00:00<00:00, 527.65it/s]

[Jan 25, 23:29:46] Loading filter_pids_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


[Jan 25, 23:29:46] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What animation studio did Miyazaki found?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  7284,  2996,  2106,  2771,  3148, 18637,  2179,
         1029,   102,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[{'content': 'In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".',
  'score': 25.90448570251465,
  'rank': 1,
  'document_id': 'miyazaki',
  'document_metadata': {'entity': 'person', 'source': 'wikipedia'}},
 {'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is widely re

But is it efficient? Let's check how long it takes ColBERT to embed our query and retrieve documents. Because ColBERT's main retrieval approach relies on `maxsim`, a very efficient operation, searching through orders of magnitudes more documents shouldn't take much longer:

In [11]:
%%timeit
RAG.search(query="What animation studio did Miyazaki found?")

/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


37.7 ms ± 486 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


You can also batch queries, which will run faster if you've got many different queries to run at once. The output format is the same as for a single query, except it's a list of lists, where item at index `i` will correspond to the query at index `i`:

In [12]:
all_results = RAG.search(query=["What animation studio did Miyazaki found?", "Miyazaki son name"], k=k)
all_results

2it [00:00, 380.38it/s]


[[{'content': 'In April 1984, Miyazaki opened his own office in Suginami Ward, naming it Nibariki.\n\n\n=== Studio Ghibli ===\n\n\n==== Early films (1985–1996) ====\nIn June 1985, Miyazaki, Takahata, Tokuma and Suzuki founded the animation production company Studio Ghibli, with funding from Tokuma Shoten. Studio Ghibli\'s first film, Laputa: Castle in the Sky (1986), employed the same production crew of Nausicaä. Miyazaki\'s designs for the film\'s setting were inspired by Greek architecture and "European urbanistic templates".',
   'score': 25.90448760986328,
   'rank': 1,
   'document_id': 'miyazaki',
   'document_metadata': {'entity': 'person', 'source': 'wikipedia'}},
  {'content': 'Hayao Miyazaki (宮崎 駿 or 宮﨑 駿, Miyazaki Hayao, Japanese: [mijaꜜzaki hajao]; born January 5, 1941) is a Japanese animator, filmmaker, and manga artist. A co-founder of Studio Ghibli, he has attained international acclaim as a masterful storyteller and creator of Japanese animated feature films, and is wid

And that's it for the basics of querying an index! You're now ready to index and retrieve documents with RAGatouille!

## Using an already-created index

In the examples above, we embedded documents into an index and queried it during the same session. But a key feature is **persistence**: indexing is the slowest part, we don't want to have to do this every-time!

Loading an already-created Index is just as straightforward as creating one from scratch. First, we'll load up an instance of RAGPretrainedModel from the index, where the full configuration of the embedder is stored:

In [13]:
# This is the path to index. We recommend keeping this path format when using RAGatouille somewhere else.
path_to_index = ".ragatouille/colbert/indexes/Miyazaki/"
RAG = RAGPretrainedModel.from_index(path_to_index)

/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


And that's it! The index is now fully ready to be queried using `search()` as above.

### Updating an index

Once you've loaded an existing index, you might want to add new documents to it. RAGatouille supports this via the `RAGPretrainedModel.add_to_index()` function. Due to the way ColBERT stores documents as bags-of-embeddings, there are cases where recreating the index is more efficient than updating it -- you don't need to worry about it, the most efficient method is automatically used when you call `add_to_index()`.

You want to expand, and cover more of Studio Ghibli, so let's get the Studio's page into our index too!

In [14]:
new_documents = get_wikipedia_page("Studio_Ghibli")

RAG.add_to_index([new_documents])

[Jan 25, 23:29:56] #> Loading codec...
[Jan 25, 23:29:56] #> Loading IVF...
[Jan 25, 23:29:56] #> Loading doclens...


100%|███████████████████████| 1/1 [00:00<00:00, 4132.32it/s]

[Jan 25, 23:29:56] #> Loading codes and residuals...



100%|████████████████████████| 1/1 [00:00<00:00, 502.73it/s]

New index_name received! Updating current index_name (Miyazaki) to Miyazaki


...

[Jan 25, 23:30:02] [0] 		 #> Encoding 59 passages..


100%|██████████| 1/1 [00:02<00:00,  2.46s/it]
WARNING clustering 6997 points to 1024 centroids: please provide at least 39936 training points
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]/Users/satyamtiwary/Library/Caches/pypoetry/virtualenvs/ragatouille-jR6OsOZ3-py3.10/lib/python3.10/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


[Jan 25, 23:30:05] [0] 		 avg_doclen_est = 124.83050537109375 	 len(local_sample) = 59
[Jan 25, 23:30:05] [0] 		 Creating 1,024 partitions.
[Jan 25, 23:30:05] [0] 		 *Estimated* 7,364 embeddings.
[Jan 25, 23:30:05] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/Miyazaki/plan.json ..
Clustering 6997 points in 128D to 1024 clusters, redo 1 times, 20 iterations
  Preprocessing in 0.00 s
[0.037, 0.037, 0.036, 0.035, 0.03, 0.037, 0.032, 0.031, 0.032, 0.035, 0.036, 0.037, 0.033, 0.032, 0.034, 0.038, 0.029, 0.032, 0.037, 0.033, 0.036, 0.035, 0.032, 0.035, 0.034, 0.035, 0.036, 0.029, 0.034, 0.032, 0.036, 0.039, 0.036, 0.03, 0.032, 0.036, 0.031, 0.036, 0.034, 0.038, 0.035, 0.037, 0.033, 0.031, 0.034, 0.031, 0.033, 0.034, 0.033, 0.034, 0.033, 0.035, 0.034, 0.032, 0.032, 0.034, 0.034, 0.037, 0.039, 0.034, 0.031, 0.034, 0.033, 0.033, 0.035, 0.033, 0.037, 0.035, 0.03, 0.036, 0.035, 0.032, 0.031, 0.035, 0.032, 0.033, 0.036, 0.035, 0.033, 0.035, 0.034, 0.034, 0.031, 0.039, 0.033, 


100%|██████████| 1/1 [00:02<00:00,  2.04s/it]
1it [00:02,  2.06s/it]
100%|██████████| 1024/1024 [00:00<00:00, 346759.83it/s]


[Jan 25, 23:30:07] #> Optimizing IVF to store map from centroids to list of pids..
[Jan 25, 23:30:07] #> Building the emb2pid mapping..
[Jan 25, 23:30:07] len(emb2pid) = 7365
[Jan 25, 23:30:07] #> Saved optimized IVF to .ragatouille/colbert/indexes/Miyazaki/ivf.pid.pt

#> Joined...
Done indexing!
Successfully updated index with 59 new documents!
 New index size: 140


And again, that's it! The index has been updated with your new document set, and the updates are already persisted to disk. You're now ready to query it with `search()`!

# Uploading and Using an Index from Huggingface


In [15]:
import os
HF_TOKEN = os.getenv("HF_TOKEN")

!huggingface-cli login --token $HF_TOKEN

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/satyamtiwary/.cache/huggingface/token
Login successful


In [16]:
repo_name = "Technoculture/Miyazaki"

from ragatouille.models.utils import upload_index_and_model

In [17]:
from pathlib import Path

# upload_index_and_model(
#     "/Users/satyamtiwary/.cache/huggingface/hub/models--colbert-ir--colbertv2.0/snapshots/c78cfaf20f3706c805646582efb1d1442f1ee4a7",
#     repo_name,
#     ".ragatouille/colbert/indexes/Miyazaki"
# )

RAG.

ColBERTConfig(query_token_id='[unused0]', doc_token_id='[unused1]', query_token='[Q]', doc_token='[D]', ncells=None, centroid_score_threshold=None, ndocs=None, load_index_with_mmap=False, index_path=None, nbits=1, kmeans_niters=20, resume=False, similarity='cosine', bsize=8, accumsteps=1, lr=1e-05, maxsteps=400000, save_every=None, warmup=20000, warmup_bert=None, relu=False, nway=64, use_ib_negatives=True, reranker=False, distillation_alpha=1.0, ignore_scores=False, model_name=None, query_maxlen=32, attend_to_mask_tokens=False, interaction='colbert', dim=128, doc_maxlen=180, mask_punctuation=True, checkpoint='/Users/satyamtiwary/.cache/huggingface/hub/models--colbert-ir--colbertv2.0/snapshots/c78cfaf20f3706c805646582efb1d1442f1ee4a7', triples='/future/u/okhattab/root/unit/experiments/2021.10/downstream.distillation.round2.2_score/round2.nway6.cosine.ib/examples.64.json', collection='/future/u/okhattab/data/MSMARCO/collection.tsv', queries='/future/u/okhattab/data/MSMARCO/queries.train.

In [4]:
!ls -a .tmp

.  ..


In [5]:
RAG = RAGPretrainedModel.from_index(index_name)

NameError: name 'RAGPretrainedModel' is not defined

In [18]:

export_path = "~/.rag"
str(Path(export_path) / "model")

'~/.rag/model'